## Prepare Data + Model

In [ ]:
# !cat examples/data/text_forward.txt

In [ ]:
# !ls -al ./outputs/en.1-percent.elmo-bert-causal-fixr2l

In [ ]:
import torch
from newlm.lm.elmo.modeling_elmo.elmo_head import ELMOBertLMHeadModel
from newlm.lm.elmo.lm_builder import ELMOLMBuilder
from transformers import BertConfig

#### Model

In [ ]:
model = ELMOBertLMHeadModel.from_pretrained(
    "./outputs/en.100-percent.elmo-bert-causal.40k"
) # use pre-trained model

In [ ]:
model.eval()
print("Model in eval mode for consistency")

#### Data

In [ ]:
%%capture

from newlm.utils.file_util import read_from_yaml
config_file = read_from_yaml('examples/configs_gcloud/run-100-percent.elmo-bert-causal.yaml')

# lm builder (helper)
elmo_lm_builder = ELMOLMBuilder(
    model_config = config_file['lm']['model']['config'],
    tokenizer="./outputs/en.100-percent.elmo-bert-causal.40k", # use pre-trained tokenizer
    model_type="bert-causal-elmo",
    max_len=128
)

# dataset-forward
train_path = "./examples/data/text_forward-small.txt"
ds_f = elmo_lm_builder._get_dataset(train_path)

In [ ]:
# trainer (helper)
from transformers import TrainingArguments, Trainer
args = TrainingArguments(output_dir="tmpout",**config_file['lm']['hf_trainer']['args'])

# dataloader-forward
trainer = Trainer(model=model, args=args, data_collator=elmo_lm_builder.data_collator, train_dataset=ds_f,)
dl_f = trainer.get_train_dataloader() # Data Loader-forward

In [ ]:
batch_f = next(iter(dl_f))
batch_f['input_ids'].shape

In [ ]:
model.eval()
print("Model in eval mode for consistency")

## Sanity Check

In [ ]:
# batch_f

In [ ]:
import torch

def reverse_batch(batch_f):
    # reverse input
    batch_f_input = torch.clone(batch_f['input_ids'])
    batch_f_rev_input = torch.cat(
        (
            batch_f_input[0][0:1],
            torch.flip(batch_f_input[0][1:-1], [0]),
            batch_f_input[0][-1:]
        )
    )
    batch_f_rev_input = batch_f_rev_input.reshape(1,-1)

    # reverse labels
    batch_f_rev_labels = torch.clone(batch_f_rev_input)
    
    # batch_rev
    batch_rev = batch_f.copy()
    batch_rev['input_ids'] = batch_f_rev_input
    batch_rev['labels'] = batch_f_rev_labels
    
    return batch_rev

In [ ]:
import pandas as pd

def pandas_check(batch_f, batch_rev):
    tokens_f = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_f['input_ids'][0])
    tokens_f_rev = elmo_lm_builder.tokenizer.convert_ids_to_tokens(batch_rev['input_ids'][0])
    return pd.DataFrame({"forward": tokens_f, "reverse": tokens_f_rev})

#### Normal vs Reverse

In [ ]:
batch_rev = reverse_batch(batch_f)

In [ ]:
pandas_check(batch_f, batch_rev)

In [ ]:
batch_f['input_ids'].shape, batch_rev['input_ids'].shape

In [ ]:
res = model(**batch_f) # forward

In [ ]:
res = model(**batch_rev) # reverse

#### Random String

In [ ]:
batch_f['input_ids'].shape

In [ ]:
# shuffle data
batch_f_input = batch_f['input_ids']
batch_shuffle_input = torch.cat(
    (
        batch_f_input[0][0:1],
        torch.randint(
            low=5, # 0-4 > ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']
            high=29999,
            size=(121,), # modified based on seqlen!
            dtype=torch.long
        ),
        batch_f_input[0][-1:]
    )
)
batch_shuffle_input = batch_shuffle_input.reshape(1,-1)
# labels
batch_shuffle_labels = torch.clone(batch_shuffle_input)    
# batch_shuffle
batch_shuffle = batch_f.copy()
batch_shuffle['input_ids'] = batch_shuffle_input
batch_shuffle['labels'] = batch_shuffle_labels

In [ ]:
batch_shuffle_rev = reverse_batch(batch_shuffle)

##### Trial-1

In [ ]:
pandas_check(batch_shuffle, batch_shuffle_rev)

In [ ]:
res = model(**batch_shuffle) # forward

In [ ]:
res = model(**batch_shuffle_rev) # reverse

##### Trial-2

In [ ]:
pandas_check(batch_shuffle, batch_shuffle_rev)

In [ ]:
res = model(**batch_shuffle) # forward

In [ ]:
res = model(**batch_shuffle_rev) # reverse

##### Trial-3

In [ ]:
pandas_check(batch_shuffle, batch_shuffle_rev)

In [ ]:
res = model(**batch_shuffle) # forward

In [ ]:
res = model(**batch_shuffle_rev) # reverse